In [3]:
import yfinance as yf
import pandas as pd

# List of 10 tech stocks
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'NFLX', 'ADBE', 'INTC']

# Download data for these stocks
intraday_data = yf.download(tickers=tech_stocks, period='5d', interval='1m')

# Save to CSV for further analysis (optional)
intraday_data.to_csv(f'tech_stocks_intraday_data.csv')


[*********************100%***********************]  10 of 10 completed


In [4]:
adj_close_data = intraday_data['Adj Close']
# Check if there are still missing values
#print(adj_close_data.isnull().sum())
# Display the first few rows
adj_close_data.fillna(method='ffill', inplace=True)
adj_close_data

# Calculate the percentage change for each stock
pct_change_data = adj_close_data.pct_change() * 100  # Convert to percentage

# Display the first few rows of percentage change data
pct_change_data.dropna(inplace=True)

pct_change_data

C:\Users\User\AppData\Local\Temp\ipykernel_2300\3940698456.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adj_close_data.fillna(method='ffill', inplace=True)


Ticker,AAPL,ADBE,AMZN,GOOGL,INTC,META,MSFT,NFLX,NVDA,TSLA
Datetime,,,,,,,,,,
2024-09-23 13:31:00+00:00,-0.068330,0.113992,-0.177188,-0.032072,0.655712,0.129216,-0.179502,0.151439,0.364314,0.783760
2024-09-23 13:32:00+00:00,-0.240421,-0.211601,-0.261035,-0.556799,-0.796207,-0.280924,-0.608642,-0.111986,-0.504829,-0.006228
2024-09-23 13:33:00+00:00,-0.006633,0.244377,0.091603,-0.213006,-0.693826,0.066909,0.104383,0.111885,0.012897,0.437499
2024-09-23 13:34:00+00:00,0.249863,-0.132793,0.026149,0.054889,-0.584172,-0.103819,-0.040547,0.141450,-0.193354,0.279407
2024-09-23 13:35:00+00:00,-0.167632,0.026587,-0.036601,-0.030415,0.061660,-0.074857,-0.148361,-0.059827,-0.112072,-0.022375
...,...,...,...,...,...,...,...,...,...,...
2024-09-27 19:55:00+00:00,0.087822,0.067956,0.021273,0.003053,-0.041728,0.034368,0.065477,-0.036679,0.000000,-0.111333
2024-09-27 19:56:00+00:00,-0.024127,-0.032013,-0.010630,-0.070144,0.062615,-0.024657,-0.045572,-0.099492,-0.007494,0.084561
2024-09-27 19:57:00+00:00,0.046076,0.029120,-0.021279,0.015257,0.062584,-0.029084,0.046763,-0.051558,-0.037792,0.001910


In [5]:
# Use quantiles to classify the data
def classify_quantiles(pct_change, q1, q2, q3, q4):
    if pct_change > q4:
        return 'A+'
    elif pct_change > q3:
        return 'B+'
    elif pct_change > q2:
        return 'C+'
    elif pct_change > q1:
        return 'C-'
    else:
        return 'B-'

# Apply quantile-based classification
classified_data_quantiles = pct_change_data.copy()

for ticker in pct_change_data.columns:
    q1 = pct_change_data[ticker].quantile(0.2)
    q2 = pct_change_data[ticker].quantile(0.4)
    q3 = pct_change_data[ticker].quantile(0.6)
    q4 = pct_change_data[ticker].quantile(0.8)
    
    classified_data_quantiles[ticker] = pct_change_data[ticker].apply(classify_quantiles, args=(q1, q2, q3, q4))

# Display classified data
print(classified_data_quantiles.head())

Ticker                    AAPL ADBE AMZN GOOGL INTC META MSFT NFLX NVDA TSLA
Datetime                                                                    
2024-09-23 13:31:00+00:00   B-   A+   B-    B-   A+   A+   B-   A+   A+   A+
2024-09-23 13:32:00+00:00   B-   B-   B-    B-   B-   B-   B-   B-   B-   C+
2024-09-23 13:33:00+00:00   C+   A+   A+    B-   B-   A+   A+   A+   C+   A+
2024-09-23 13:34:00+00:00   A+   B-   B+    A+   B-   B-   B-   A+   B-   A+
2024-09-23 13:35:00+00:00   B-   B+   B-    B-   B+   B-   B-   B-   B-   C-


In [11]:
# Modified function to create variable-length sequences as lists of tokens
def create_variable_sequences(data):
    sequences = {}  # Dictionary to store sequences for each stock
    for ticker in data.columns:  # Iterate through each stock (each column)
        stock_data = data[ticker]  # Extract the column for the stock
        current_seq = []  # Temporary list to store the current sequence
        stock_sequences = []  # List to store all sequences for this stock
        
        for i in range(len(stock_data)):
            if i == 0 or stock_data.iloc[i] == stock_data.iloc[i-1]:
                current_seq.append(stock_data.iloc[i])  # Continue the sequence if the trend is the same
            else:
                # Append the current sequence (as a list of tokens)
                stock_sequences.append(current_seq)
                current_seq = [stock_data.iloc[i]]  # Start a new sequence
        
        if current_seq:
            stock_sequences.append(current_seq)  # Append the final sequence

        sequences[ticker] = stock_sequences  # Store the sequences for the stock

    return sequences

# Example: Generating sequences for AAPL
variable_sequences = create_variable_sequences(classified_data_quantiles)

# Display the variable-length sequences for AAPL (as lists of tokens)
print("Variable Length Sequences for AAPL:", variable_sequences['AAPL'][:5])

Variable Length Sequences for AAPL: [['B-', 'B-'], ['C+'], ['A+'], ['B-'], ['A+', 'A+', 'A+', 'A+']]


In [17]:
variable_sequences

{'AAPL': [['B-', 'B-'],
  ['C+'],
  ['A+'],
  ['B-'],
  ['A+', 'A+', 'A+', 'A+'],
  ['B-', 'B-', 'B-', 'B-'],
  ['A+'],
  ['B+'],
  ['B-'],
  ['A+', 'A+', 'A+'],
  ['B-', 'B-'],
  ['A+'],
  ['B-'],
  ['C-'],
  ['C+'],
  ['B-'],
  ['A+'],
  ['B-', 'B-'],
  ['C+'],
  ['A+'],
  ['C-'],
  ['B-', 'B-'],
  ['A+'],
  ['C-'],
  ['B-'],
  ['C-', 'C-'],
  ['A+', 'A+'],
  ['B-', 'B-'],
  ['A+', 'A+'],
  ['C-', 'C-'],
  ['A+', 'A+'],
  ['B+'],
  ['A+', 'A+', 'A+'],
  ['C-'],
  ['B-'],
  ['C-', 'C-'],
  ['B+'],
  ['A+'],
  ['C+', 'C+'],
  ['C-', 'C-'],
  ['B-'],
  ['C-'],
  ['A+'],
  ['C-'],
  ['B-'],
  ['C+'],
  ['A+'],
  ['B+'],
  ['A+', 'A+'],
  ['C-'],
  ['A+'],
  ['B+'],
  ['A+', 'A+', 'A+'],
  ['C-'],
  ['B+'],
  ['A+', 'A+'],
  ['B-', 'B-', 'B-'],
  ['C+', 'C+'],
  ['C-'],
  ['C+'],
  ['B+'],
  ['A+'],
  ['B+'],
  ['B-', 'B-'],
  ['A+'],
  ['B-'],
  ['A+'],
  ['B-'],
  ['B+'],
  ['C-'],
  ['B+', 'B+'],
  ['B-', 'B-'],
  ['C-', 'C-'],
  ['A+'],
  ['B+'],
  ['C+'],
  ['C-'],
  ['A+', 'A+'],
  

In [48]:
import nltk
from nltk import CFG

# Define the CFG for stock movements, considering all transitions
grammar = CFG.fromstring("""
    S -> A S | B S | C S | ''
    A -> 'A+' A | 'A-' A | ''
    B -> 'B+' B | 'B-' B | ''
    C -> 'C+' C | 'C-' C | ''
""")

In [49]:
# Initialize the parser with the CFG
parser = nltk.ChartParser(grammar)

# Function to parse and display the parse tree for all sequences in all stocks
def parse_all_stocks(variable_sequences, parser, max_sequences=None):
    """
    Parses all sequences for all stocks in variable_sequences using the provided parser.
    
    Args:
        variable_sequences (dict): Dictionary where each key is a stock and value is a list of tokenized sequences.
        parser (nltk.ChartParser): The CFG parser.
        max_sequences (int): Maximum number of sequences to parse per stock. If None, parse all.
    
    Returns:
        parsed_trees (dict): A dictionary where each stock key contains a list of parsed trees.
    """
    parsed_trees = {}  # Dictionary to store parsed trees for each stock

    for stock, sequences in variable_sequences.items():
        print(f"\nParsing sequences for stock: {stock}")
        stock_parsed_trees = []  # List to store parsed trees for this stock
        
        # Limit the number of sequences to parse if max_sequences is set
        num_sequences = len(sequences) if max_sequences is None else min(len(sequences), max_sequences)
        
        for seq in sequences[:num_sequences]:
            print(f"Sequence: {seq}")
            # Parse the sequence (already tokenized) and store the trees

            # Check the tokens in the sequence
            if all(token in ['A+', 'A-', 'B+', 'B-', 'C+', 'C-'] for token in seq):
                # Parse the sequence (already tokenized) and store the trees
                for tree in parser.parse(seq):
                    stock_parsed_trees.append(tree)  # Append the parsed tree
                    print(tree)  # Optionally print the tree for inspection
            else:
                print(f"Invalid tokens in sequence: {seq}")
        
        parsed_trees[stock] = stock_parsed_trees  # Store the parsed trees for this stock

    return parsed_trees

# Parse and display the sequences for AAPL
parsed_trees = parse_all_stocks(variable_sequences, parser, max_sequences=10)
parsed_trees


Parsing sequences for stock: AAPL
Sequence: ['B-', 'B-']
Sequence: ['C+']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['A+', 'A+', 'A+', 'A+']
Sequence: ['B-', 'B-', 'B-', 'B-']
Sequence: ['A+']
Sequence: ['B+']
Sequence: ['B-']
Sequence: ['A+', 'A+', 'A+']

Parsing sequences for stock: ADBE
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['B+', 'B+']
Sequence: ['B-']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['A+']
Sequence: ['C+']

Parsing sequences for stock: AMZN
Sequence: ['B-', 'B-']
Sequence: ['A+']
Sequence: ['B+']
Sequence: ['B-']
Sequence: ['A+', 'A+', 'A+']
Sequence: ['C-']
Sequence: ['B-', 'B-', 'B-', 'B-']
Sequence: ['A+', 'A+']
Sequence: ['B-']
Sequence: ['A+', 'A+', 'A+']

Parsing sequences for stock: GOOGL
Sequence: ['B-', 'B-', 'B-']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['C+']
Sequence: ['A+']
Sequence: ['B-']
Sequence: ['C-', 'C-']
Sequence: ['C+']

Parsing sequences for stock: INTC
Sequence: ['

{'AAPL': [],
 'ADBE': [],
 'AMZN': [],
 'GOOGL': [],
 'INTC': [],
 'META': [],
 'MSFT': [],
 'NFLX': [],
 'NVDA': [],
 'TSLA': []}

In [50]:
import nltk
from nltk import CFG

# Define a CFG to handle sequences of movements with repetitions and transitions
grammar = CFG.fromstring("""
    S -> A S | B S | C S | ''
    A -> 'A+' A | 'A-' A | ''
    B -> 'B+' B | 'B-' B | ''
    C -> 'C+' C | 'C-' C | ''
""")

# Initialize the parser with the updated CFG
parser = nltk.ChartParser(grammar)

# Example sequences from the data
test_sequences = [
    ['B-', 'B-'],
    ['A+', 'A+', 'A+', 'A+'],
    ['C+'],
    ['A+', 'B-', 'C+'],
    ['B+', 'B+', 'B-'],
    ['A+', 'A+', 'B-', 'C+'],
]

# Function to parse and print parse trees for sequences
def parse_and_print(test_sequences, parser):
    for seq in test_sequences:
        print(f"\nParsing sequence: {seq}")
        for tree in parser.parse(seq):
            print(tree)

# Parse and display the sequences
parse_and_print(test_sequences, parser)




Parsing sequence: ['B-', 'B-']

Parsing sequence: ['A+', 'A+', 'A+', 'A+']

Parsing sequence: ['C+']

Parsing sequence: ['A+', 'B-', 'C+']

Parsing sequence: ['B+', 'B+', 'B-']

Parsing sequence: ['A+', 'A+', 'B-', 'C+']
